In [1]:
import pandas as pd
from statistics import mean
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [2]:
links_df = pd.read_csv('data/links.csv')
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')
tags_df = pd.read_csv('data/tags.csv')

In [3]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
genres = {"Action": [],
          "Adventure": [],
          "Animation": [],
          "Children": [],
          "Comedy": [],
          "Crime": [],
          "Documentary": [],
          "Drama": [],
          "Fantasy": [],
          "Film-Noir": [],
          "Horror": [],
          "Musical": [],
          "Mystery": [],
          "Romance": [],
          "Sci-Fi": [],
          "Thriller": [],
          "War": [],
          "Western": []}

for index, row in movies_df.iterrows():
    current_movie_genres = row.genres.split('|')
    
    for key in genres.keys():
        genres[key].append(1 if key in current_movie_genres else 0)

for key in genres.keys():
    movies_df[key] = genres[key]
    
movies_df = movies_df.drop(columns="genres")

In [5]:
ratings_df.head()

movie_ids = links_df.movieId.values

average_ratings = []

for movie_id in movie_ids:
    ratings = ratings_df.loc[ratings_df['movieId'] == movie_id].rating.values
    
    average_ratings.append(mean(ratings) if len(ratings) != 0 else 0)
    
movies_df.insert(2, 'avg_rating', average_ratings)

In [6]:
movies_df.head()

,movieId,title,avg_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),3.920930,0,1,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),3.431818,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),3.259615,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),2.357143,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),3.071429,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# group tags by movieid, use count vectorizer with stop words

tags = []

for movieid in movies_df.movieId.values:
    tags.append(' '.join(tags_df.loc[tags_df.movieId == movieid].tag.values))
    
movies_df['tags'] = tags

movies_df.head()

,movieId,title,avg_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,tags
0,1,Toy Story (1995),3.920930,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,pixar pixar fun
1,2,Jumanji (1995),3.431818,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,fantasy magic board game Robin Williams game
2,3,Grumpier Old Men (1995),3.259615,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,moldy old
3,4,Waiting to Exhale (1995),2.357143,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,
4,5,Father of the Bride Part II (1995),3.071429,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,pregnancy remake


In [48]:
transformer = TfidfTransformer()
x = transformer.fit_transform(movies_df.tags.values)

ValueError: could not convert string to float: 'funny'

In [45]:
clusters = 6
model = KMeans(n_clusters=clusters, max_iter=500, tol=0.00001, random_state=222)
model.fit(x)

centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = transformer.get_feature_names()
for i in range(clusters):
    print('cluster {}:'.format(i), end='')
    for j in centroids[i, :3]:
        print(' {}'.format(terms[j]), end='')
    print('')

NameError: name 'transformer' is not defined